In [2]:
# # Testing the llm proxy
# import os
# # pem_path = "/opt/homebrew/etc/openssl@3/certs/../cert.pem"
# pem_path = "/opt/homebrew/etc/openssl@3/cert.pem"
# # Ensure the environment variables are set before making the API call
# os.environ['REQUESTS_CA_BUNDLE'] = pem_path
# os.environ['SSL_CERT_FILE'] = pem_path

# # Proceed with the API call
# import getpass
# from openai import OpenAI
# client = OpenAI(
#     base_url = os.getenv('proxyllmendpoint'),
#     api_key  = os.getenv('proxyllmuserkey'), 
# )
# completion =  client.chat.completions.create(
#                     model    = "gpt-4o-mini",
#                     messages = [{ "role"   : "user",
#                                   "content": "Write a function that prints n primes in python"}],
#                     user     = getpass.getuser() # DO NOT HARDCODE A USER HERE
#                 )

# import os
# import getpass
# pem_path = "/opt/homebrew/etc/openssl@3/certs/../cert.pem"
# # Ensure the environment variables are set before making the API call
# os.environ['REQUESTS_CA_BUNDLE'] = pem_path
# # os.environ['SSL_CERT_FILE']      = pem_path

# # Proceed with the API call
# from openai import OpenAI
# client = OpenAI(
#     base_url = os.getenv('proxyllmendpoint'),
#     api_key  = os.getenv('proxyllmappkey'), # DO NOT HARDCODE YOUR KEY
# )
# completion =  client.chat.completions.create(
#                     model    = "gpt-4o-mini",
#                     messages = [{ "role"   : "user",
#                                   "content": "Write a function that prints n primes in python"}],
#                     # user     = "prragenticworkflow" # DO NOT HARDCODE A USER HERE
#                     user = getpass.getuser()
#                 )

### Importing Required Libraries

In [8]:
import os
import pandas as pd
import getpass # to get the current user
from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationChain
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from pydantic.v1 import BaseModel, Field
from typing import Optional, TypedDict, Annotated
import operator
from semantic_router.utils.function_call import FunctionSchema





/Users/akhilred/Library/Python/3.13/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Defining file read tool

In [9]:
def read_python_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "Error: File not found."
    
class codepath(BaseModel):
  path: str = Field(description="code path to execute")

@tool(args_schema = codepath)
def execute_query(path: str) -> str:
  """Returns the result of code path execution"""
  return read_python_file(path)

In [10]:
# import requests
# # Function to fetch repository contents from GitHub
# def get_repo_contents(repo_owner, repo_name, access_token):
#     """
#     Fetch the contents of a repository from GitHub.

#     :param repo_owner: Owner of the repository
#     :param repo_name: Name of the repository
#     :param access_token: Personal access token for GitHub
#     :return: Repository contents as a list of files and directories
#     """
#     url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents"

#     headers = {
#         "Authorization": f"token {access_token}",
#         "Accept": "application/vnd.github.v3+json"
#     }
#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         return response.json()
#     else:
#         return f"Error: Unable to fetch repository contents (Status Code: {response.status_code})"

# # Example usage
# repo_owner = "akhilntap"
# repo_name = "ProductionReadinessAgenticAI"
# access_token = os.environ.get("githubpat")
# repo_contents = get_repo_contents(repo_owner, repo_name, access_token)
# print(repo_contents)
# repo_table = pd.DataFrame(repo_contents)
# repo_table

In [11]:
# def fetch_github_file_content(repo_contents, file_name):
#     """
#     Fetch the content of a specific file from the GitHub repository.

#     :param repo_contents: List of repository contents
#     :param file_name: Name of the file to fetch
#     :return: File content as a string or an error message
#     """
#     # Find the file in the repository contents
#     file_info = next((item for item in repo_contents if item['name'] == file_name), None)
    
#     if not file_info:
#         return f"Error: File '{file_name}' not found in the repository."
    
#     if not file_info.get('download_url'):
#         return f"Error: File '{file_name}' does not have a downloadable URL."
    
#     # Fetch the file content
#     response = requests.get(file_info['download_url'])
    
#     if response.status_code == 200:
#         return response.text
#     else:
#         return f"Error: Unable to fetch file content (Status Code: {response.status_code})"

# # Example usage
# file_name = "40poc.ipynb" 
# file_content = fetch_github_file_content(repo_contents, file_name)
# print(file_content)

In [14]:

#when you write code for code diff pull based on pull request, modify this function with good way of representing parameters
def fetch_file_from_github(file_name, repo_owner="akhilntap", repo_name="ProductionReadinessAgenticAI", access_token=os.environ.get("githubpat")):
    """
    Fetch the content of a specific file from a GitHub repository.

    :param repo_owner: Owner of the repository
    :param repo_name: Name of the repository
    :param access_token: Personal access token for GitHub
    :param file_name: Name of the file to fetch
    :return: File content as a string or an error message
    """
    # Fetch repository contents
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents"
    headers = {
        "Authorization": f"token {access_token}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return f"Error: Unable to fetch repository contents (Status Code: {response.status_code})"
    
    repo_contents = response.json()

    # Find the file in the repository contents
    file_info = next((item for item in repo_contents if item['name'] == file_name), None)
    
    if not file_info:
        return f"Error: File '{file_name}' not found in the repository."
    
    if not file_info.get('download_url'):
        return f"Error: File '{file_name}' does not have a downloadable URL."
    
    # Fetch the file content
    file_response = requests.get(file_info['download_url'])
    
    if file_response.status_code == 200:
        return file_response.text
    else:
        return f"Error: Unable to fetch file content (Status Code: {file_response.status_code})"
    
class githubfile(BaseModel):
  path: str = Field(description="github file to fetch")

@tool(args_schema = githubfile)
def execute_githubpull(file_name: str) -> str:
  """Pull the code contents from github"""
  return fetch_file_from_github(file_name)

### defining agent state

In [15]:
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], operator.add]

In [16]:
# create the function calling schema for ollama
execute_query_schema = FunctionSchema(read_python_file).to_ollama()
# execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema

{'type': 'function',
 'function': {'name': 'read_python_file',
  'description': 'None',
  'parameters': {'type': 'object',
   'properties': {'file_path': {'description': None, 'type': 'object'},
    'description': None},
   'required': []}}}

In [18]:
class codeAgent:
  # initialising the object
  def __init__(self, model, tools, system_prompt = ""):
    self.system_prompt = system_prompt

    # initialising graph with a state 
    graph = StateGraph(AgentState)

    # adding nodes 
    graph.add_node("llm", self.call_llm)
    graph.add_node("function", self.execute_function)
    graph.add_conditional_edges(
      "llm",
      self.exists_function_calling,
      {True: "function", False: END}
    )
    graph.add_edge("function", "llm")

    # setting starting point
    graph.set_entry_point("llm")

    self.graph = graph.compile()
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def call_llm(self, state: AgentState):
    messages = state['messages']
    # adding system prompt if it's defined
    if self.system_prompt:
        messages = [SystemMessage(content=self.system_prompt)] + messages

    # calling LLM
    message = self.model.invoke(messages)

    return {'messages': [message]}
  
  def execute_function(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls

    results = []
    for tool in tool_calls:
      # checking whether tool name is correct
      if not tool['name'] in self.tools:
        # returning error to the agent 
        result = "Error: There's no such tool, please, try again" 
      else:
        # getting result from the tool
        result = self.tools[tool['name']].invoke(tool['args'])

      results.append(
        ToolMessage(
          tool_call_id=tool['id'], 
          name=tool['name'], 
          content=str(result)
        )
    )
    return {'messages': results}
  
  def exists_function_calling(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0


# from langchain_ollama import ChatOllama
# model = ChatOllama(model="llama3.2:1b")
# # model = ChatOllama(model="codellama:latest")

# system prompt
# prompt = '''You are a senior expert in reviewing python code. 
# So, you can help the team to review the code and provide feedback. 
# You are very accurate and take into account all the nuances in code.
# Your goal is to provide the detailed documentation for any security issues in the code that will help users.'''

# prompt = '''You are a senior expert in reviewing code for observability. The best one that exists.
# Your goal is to analyze the code on the following questions 
# 1. Are there actionable alerts identified for the feature? Are there Runbooks for the actionable alerts? Do we have TSGs attached to the alert?
# 2. Add metrics to monitor dependencies and exception handling on components, infrastructure and features so that SRE can create alerts to reduce TTD?
# 3. Are there CorrelationIDs established in logs to derive error lineage across various components?
# 4. Can the feature/service support changing log levels for troubleshooting purposes?
# 5. Are there critical log lines that we need to get alerted upon?
# Provide response in the format as follows: {question: response}
# '''

prompt = '''You are a senior expert in reviewing code for resiliency. The best one that exists.
Your goal is to analyze the code on the following questions 
1. Can the service/feature sustain a single node/pod failure?
2. Can the feature/service recover gracefully without requiring a restart?
3. Are there conditions that will require a human intervention to start the pods? if so, what are they?
4. What will be the customer impact(if any) due to lack of high availability?
Provide response in the format as follows: {question: response}
'''

#  getting the required ssl certificates
pem_path = "/opt/homebrew/etc/openssl@3/cert.pem"
# Ensure the environment variables are set before making the API call
os.environ['REQUESTS_CA_BUNDLE'] = pem_path
os.environ['SSL_CERT_FILE'] = pem_path


model = ChatOpenAI(model_name      = "gpt-4o-mini",
                 openai_api_base = os.getenv('proxyllmendpoint'),
                 openai_api_key  = os.getenv('proxyllmuserkey'),
                 model_kwargs    = {'user': getpass.getuser() })

doc_agent = codeAgent(model, [execute_query], system_prompt=prompt)

# messages = [HumanMessage(content="the code is in the path '/Users/akhilred/Desktop/Billing Pyton Script/Billing_Usage_Extraction.py'. Analyze the code for resiliency requirements mentioned in the prompt")]
messages = [HumanMessage(content="Analyze the code with name 40poc.ipynb from github for resiliency requirements mentioned in the prompt")]
result = doc_agent.graph.invoke({"messages": messages})

2025-04-17 18:04:35 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://llm-proxy-api.ai.openeng.netapp.com/chat/completions "HTTP/1.1 200 OK"
2025-04-17 18:04:38 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://llm-proxy-api.ai.openeng.netapp.com/chat/completions "HTTP/1.1 200 OK"


In [19]:
print(result['messages'][-1].content)

Based on the analysis of the code from `40poc.ipynb`, here are the responses to the resiliency requirements:

1. **Can the service/feature sustain a single node/pod failure?** 
   - **Response:** The code does not contain any mechanisms for handling failures or redistributing tasks in the event of a single node or pod failure. In a production environment, this could lead to downtime or degraded service, indicating a lack of built-in resilience.

2. **Can the feature/service recover gracefully without requiring a restart?**
   - **Response:** The code lacks error-handling provisions that would allow it to recover gracefully after encountering errors. Currently, it returns errors without any logic to retry or handle them, which means it requires either human intervention or a restart to recover from failures.

3. **Are there conditions that will require human intervention to start the pods? if so, what are they?**
   - **Response:** Conditions that might require human intervention includ